In [1]:
import pandas as pd
import geopandas as gpd
import sqlite3

from pyogrio import geopandas


# Load required data sources

In [10]:
cnx = sqlite3.connect('data/cleaned_db.sqlite')

crime_data = pd.read_sql_query("Select * FROM crime_data", cnx)
la_data = pd.read_sql_query("Select * FROM la311_data", cnx)

# ZIP Data
zip_data = geopandas.read_file('data/zip_codes.shp')

DatabaseError: Execution failed on sql 'Select * FROM crime_data': no such table: crime_data

In [12]:
population_data = pd.read_csv("C:\\Users\\Eric\\Projects\\MADE\\made-project\\local\\population_by_zip_code.csv")

population_data = population_data.iloc[1:]
population_data['NAME'] = population_data['NAME'].map(lambda x: x.lstrip('ZCTA5 '))
population_data = population_data[['NAME', 'P1_001N']]
population_data[population_data['NAME'] == '90046'].head()

,NAME,P1_001N
49,90046,49987
